# This is the main notebook file for the machine learning program

In [1]:
# Imports
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn import datasets, linear_model
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
%run read_csv_files.ipynb

2023-02-15 14:10:35.343187: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Scaled data where the target value predicts whether the home team will win the baksetball game (Signal = '1')

## Part 1: Neural Network with Deep Learning
### 10 seasons of data from 2011 to 2021
### Three hidden layers with LeakyReLU activation and maximum nodes, one outer layer with linear activation, MSE as the loss activation, and a maximum of 80 epochs

In [2]:
# Read in the 2011-2021 dataframe from the `read_csv_files.ipynb` file
# Drop the 'Date' column from the dataframe
ten_seasons_nn = df_combined_2011_2021.drop(
    columns=['Date']
)

# Display the dataframe
ten_seasons_nn

,Home_Team,Away_Team,Over_Under,Spread,Moneyline_Home_Team,Moneyline_Away_Team,Game_Total_Points,Win_Margin
0,New York Knicks,Boston Celtics,195.5,5.0,-210.0,185.0,210.0,2.0
1,Dallas Mavericks,Miami Heat,190.5,3.5,185.0,-210.0,199.0,-11.0
2,Los Angeles Lakers,Chicago Bulls,186.0,1.5,205.0,-245.0,175.0,-1.0
3,Oklahoma City Thunder,Orlando Magic,197.0,7.5,-380.0,310.0,186.0,8.0
4,Golden State Warriors,Los Angeles Clippers,212.0,4.0,175.0,-200.0,191.0,-19.0
...,...,...,...,...,...,...,...,...
12576,Phoenix Suns,Milwaukee Bucks,221.0,5.5,-180.0,160.0,226.0,10.0
12577,Milwaukee Bucks,Phoenix Suns,222.0,3.5,-210.0,180.0,220.0,20.0
12578,Milwaukee Bucks,Phoenix Suns,220.5,3.5,-190.0,170.0,212.0,6.0
12579,Phoenix Suns,Milwaukee Bucks,218.5,4.0,-170.0,150.0,242.0,-4.0


In [3]:
# Review the data types
ten_seasons_nn.dtypes

Home_Team               object
Away_Team               object
Over_Under             float64
Spread                 float64
Moneyline_Home_Team    float64
Moneyline_Away_Team    float64
Game_Total_Points      float64
Win_Margin             float64
dtype: object

In [4]:
# Create a list of the categorical variables
categorical_variables_1 = list(
    ten_seasons_nn.dtypes[ten_seasons_nn.dtypes == 'object'].index
)

# Display the list
categorical_variables_1

['Home_Team', 'Away_Team']

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(
    sparse = False
)

In [6]:
# Encode the categorical variables into numerical format
encoded_data_1 = enc.fit_transform(
    ten_seasons_nn[categorical_variables_1]
)

In [7]:
# Create a dataframe with the encoded data
encoded_ten_seasons_df = pd.DataFrame(
    encoded_data_1,
    columns = enc.get_feature_names_out(
        categorical_variables_1
    )
)

# Display the dataframe
encoded_ten_seasons_df

,Home_Team_Atlanta Hawks,Home_Team_Boston Celtics,Home_Team_Brooklyn Nets,Home_Team_Charlotte Bobcats,Home_Team_Chicago Bulls,Home_Team_Cleveland Cavaliers,Home_Team_Dallas Mavericks,Home_Team_Denver Nuggets,Home_Team_Detroit Pistons,Home_Team_Golden State Warriors,...,Away_Team_Oklahoma City Thunder,Away_Team_Orlando Magic,Away_Team_Philadelphia 76ers,Away_Team_Phoenix Suns,Away_Team_Portland Trail Blazers,Away_Team_Sacramento Kings,Away_Team_San Antonio Spurs,Away_Team_Toronto Raptors,Away_Team_Utah Jazz,Away_Team_Washington Wizards
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
12578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
12579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Create a new variable for the exisiting numerical data
numerical_variables_1 = ten_seasons_nn.drop(
    columns = categorical_variables_1
)

# Display the dataframe
numerical_variables_1

,Over_Under,Spread,Moneyline_Home_Team,Moneyline_Away_Team,Game_Total_Points,Win_Margin
0,195.5,5.0,-210.0,185.0,210.0,2.0
1,190.5,3.5,185.0,-210.0,199.0,-11.0
2,186.0,1.5,205.0,-245.0,175.0,-1.0
3,197.0,7.5,-380.0,310.0,186.0,8.0
4,212.0,4.0,175.0,-200.0,191.0,-19.0
...,...,...,...,...,...,...
12576,221.0,5.5,-180.0,160.0,226.0,10.0
12577,222.0,3.5,-210.0,180.0,220.0,20.0
12578,220.5,3.5,-190.0,170.0,212.0,6.0
12579,218.5,4.0,-170.0,150.0,242.0,-4.0


In [9]:
# Add the numerical data to the categorical encoded data dataframe
encoded_ten_seasons_df = pd.concat(
    [encoded_ten_seasons_df,
     numerical_variables_1],
    axis=1
)

# Display the dataframe
encoded_ten_seasons_df

,Home_Team_Atlanta Hawks,Home_Team_Boston Celtics,Home_Team_Brooklyn Nets,Home_Team_Charlotte Bobcats,Home_Team_Chicago Bulls,Home_Team_Cleveland Cavaliers,Home_Team_Dallas Mavericks,Home_Team_Denver Nuggets,Home_Team_Detroit Pistons,Home_Team_Golden State Warriors,...,Away_Team_San Antonio Spurs,Away_Team_Toronto Raptors,Away_Team_Utah Jazz,Away_Team_Washington Wizards,Over_Under,Spread,Moneyline_Home_Team,Moneyline_Away_Team,Game_Total_Points,Win_Margin
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,195.5,5.0,-210.0,185.0,210.0,2.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,190.5,3.5,185.0,-210.0,199.0,-11.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,186.0,1.5,205.0,-245.0,175.0,-1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,197.0,7.5,-380.0,310.0,186.0,8.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,212.0,4.0,175.0,-200.0,191.0,-19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,221.0,5.5,-180.0,160.0,226.0,10.0
12577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,222.0,3.5,-210.0,180.0,220.0,20.0
12578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,220.5,3.5,-190.0,170.0,212.0,6.0
12579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,218.5,4.0,-170.0,150.0,242.0,-4.0


In [10]:
# Add a "Signal" column which tracks the "Win_Margin" column
encoded_ten_seasons_df['Win_Margin_Signal'] = 0.0

# When the "Win_Margin" is a greater than or equal to 0, set signal to 1
encoded_ten_seasons_df.loc[(encoded_ten_seasons_df['Win_Margin'] >= 0), 'Win_Margin_Signal'] = 1

# When the "Win_Margin" is less than 0, set signal to -1
encoded_ten_seasons_df.loc[(encoded_ten_seasons_df['Win_Margin'] < 0), 'Win_Margin_Signal'] = -1

# Display the dataframe
encoded_ten_seasons_df

,Home_Team_Atlanta Hawks,Home_Team_Boston Celtics,Home_Team_Brooklyn Nets,Home_Team_Charlotte Bobcats,Home_Team_Chicago Bulls,Home_Team_Cleveland Cavaliers,Home_Team_Dallas Mavericks,Home_Team_Denver Nuggets,Home_Team_Detroit Pistons,Home_Team_Golden State Warriors,...,Away_Team_Toronto Raptors,Away_Team_Utah Jazz,Away_Team_Washington Wizards,Over_Under,Spread,Moneyline_Home_Team,Moneyline_Away_Team,Game_Total_Points,Win_Margin,Win_Margin_Signal
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,195.5,5.0,-210.0,185.0,210.0,2.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,190.5,3.5,185.0,-210.0,199.0,-11.0,-1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,186.0,1.5,205.0,-245.0,175.0,-1.0,-1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,197.0,7.5,-380.0,310.0,186.0,8.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,212.0,4.0,175.0,-200.0,191.0,-19.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,221.0,5.5,-180.0,160.0,226.0,10.0,1.0
12577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,222.0,3.5,-210.0,180.0,220.0,20.0,1.0
12578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,220.5,3.5,-190.0,170.0,212.0,6.0,1.0
12579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,218.5,4.0,-170.0,150.0,242.0,-4.0,-1.0


In [11]:
# Change the 'Win_Margin_Signal' column to float data type
encoded_ten_seasons_df['Win_Margin_Signal'] = encoded_ten_seasons_df['Win_Margin_Signal'].astype(float)

# Verify that the data types are correct
encoded_ten_seasons_df.dtypes

Home_Team_Atlanta Hawks        float64
Home_Team_Boston Celtics       float64
Home_Team_Brooklyn Nets        float64
Home_Team_Charlotte Bobcats    float64
Home_Team_Chicago Bulls        float64
                                ...   
Moneyline_Home_Team            float64
Moneyline_Away_Team            float64
Game_Total_Points              float64
Win_Margin                     float64
Win_Margin_Signal              float64
Length: 69, dtype: object

In [12]:
# Display the amount of each 'Win_Margin_Signal' type
encoded_ten_seasons_df['Win_Margin_Signal'].value_counts()

 1.0    7318
-1.0    5263
Name: Win_Margin_Signal, dtype: int64

In [13]:
# Set the y (target value) to 'Win_Margin_Signal' column
y = encoded_ten_seasons_df['Win_Margin_Signal']

# Set the X (features) to all columns except 'Win_Margin_Signal'
X = encoded_ten_seasons_df.drop(columns=['Win_Margin_Signal'])

# Display a sample of each dataframe
display(y[:5])
display(X.head())

0    1.0
1   -1.0
2   -1.0
3    1.0
4   -1.0
Name: Win_Margin_Signal, dtype: float64

,Home_Team_Atlanta Hawks,Home_Team_Boston Celtics,Home_Team_Brooklyn Nets,Home_Team_Charlotte Bobcats,Home_Team_Chicago Bulls,Home_Team_Cleveland Cavaliers,Home_Team_Dallas Mavericks,Home_Team_Denver Nuggets,Home_Team_Detroit Pistons,Home_Team_Golden State Warriors,...,Away_Team_San Antonio Spurs,Away_Team_Toronto Raptors,Away_Team_Utah Jazz,Away_Team_Washington Wizards,Over_Under,Spread,Moneyline_Home_Team,Moneyline_Away_Team,Game_Total_Points,Win_Margin
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,195.5,5.0,-210.0,185.0,210.0,2.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,190.5,3.5,185.0,-210.0,199.0,-11.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,186.0,1.5,205.0,-245.0,175.0,-1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,197.0,7.5,-380.0,310.0,186.0,8.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,212.0,4.0,175.0,-200.0,191.0,-19.0


In [14]:
# Split the data into training and testing variables
X_train, X_test, y_train, y_test_1 = train_test_split(
    X,
    y,
    random_state=1
)

In [15]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled_1 = X_scaler.transform(X_train)
X_test_scaled_1 = X_scaler.transform(X_test)

In [16]:
# Define the amount of outputs
outputs = 1

# Display the amount of outputs
print(f"The amount of outputs for ALL the models tested in this file will be {outputs}.")

The amount of outputs for ALL the models tested in this file will be 1.


In [17]:
# Define the number of inputs to the model
inputs_1 = len(X_train.iloc[0])

# Display the amount of inputs
print(f"The amount of inputs is {inputs_1}.")

# Define the first hidden layer
hidden_1_layer_1 = ((inputs_1 + 1) // 2) + 33

# Display the amount of nodes
print(f"The amount of nodes in the first hidden layer is {hidden_1_layer_1}.")

# Define the second hidden layer
hidden_1_layer_2 = ((hidden_1_layer_1 + 1) // 2) + 10

# Display the amount of nodes
print(f"The amount of nodes in the second hidden layer is {hidden_1_layer_2}.")

# Define the third hidden layer
hidden_1_layer_3 = ((hidden_1_layer_2 + 1) // 2)

# Display the amount of nodes
print(f"The amount of nodes in the third hidden layer is {hidden_1_layer_3}.")

The amount of inputs is 68.
The amount of nodes in the first hidden layer is 67.
The amount of nodes in the second hidden layer is 44.
The amount of nodes in the third hidden layer is 22.


In [18]:
# Create a Sequential mode instance
neural_network_1 = Sequential()

2023-02-15 14:10:58.238432: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
# Add the hidden layers
# First layer
neural_network_1.add(
    Dense(
        units=hidden_1_layer_1,
        input_dim=inputs_1,
        activation='LeakyReLU'
    )
)

# Second layer
neural_network_1.add(
    Dense(
        units=hidden_1_layer_2,
        activation='LeakyReLU'
    )
)

# Third layer
neural_network_1.add(
    Dense(
        units=hidden_1_layer_3,
        activation='LeakyReLU'
    )
)

# Add the output layer
neural_network_1.add(
    Dense(
        units=outputs,
        activation='linear'
    )
)

In [20]:
# Display the Sequential model summary
neural_network_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 67)                4623      
                                                                 
 dense_1 (Dense)             (None, 44)                2992      
                                                                 
 dense_2 (Dense)             (None, 22)                990       
                                                                 
 dense_3 (Dense)             (None, 1)                 23        
                                                                 
Total params: 8,628
Trainable params: 8,628
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Add a callback to prevent overfitting the model
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=10,
    verbose=0,
    mode="min",
)

In [22]:
# Compile the Sequential model
neural_network_1.compile(
    loss="mse",
    optimizer="adam", 
    metrics=["accuracy"]
)

In [23]:
# Fit the model using 80 epochs
fit_model = neural_network_1.fit(
    X_train_scaled_1,
    y_train,
    epochs=80,
    validation_split=0.1,
    callbacks=[early_stopping]
)

Epoch 1/80
266/266 [==============================] - 5s 9ms/step - loss: 0.4648 - accuracy: 0.3321 - val_loss: 0.3489 - val_accuracy: 0.3750
Epoch 2/80
266/266 [==============================] - 3s 10ms/step - loss: 0.2710 - accuracy: 0.4215 - val_loss: 0.2004 - val_accuracy: 0.4767
Epoch 3/80
266/266 [==============================] - 2s 9ms/step - loss: 0.1268 - accuracy: 0.5169 - val_loss: 0.0878 - val_accuracy: 0.5064
Epoch 4/80
266/266 [==============================] - 2s 8ms/step - loss: 0.0626 - accuracy: 0.5518 - val_loss: 0.0659 - val_accuracy: 0.5381
Epoch 5/80
266/266 [==============================] - 2s 8ms/step - loss: 0.0392 - accuracy: 0.5661 - val_loss: 0.0490 - val_accuracy: 0.5392
Epoch 6/80
266/266 [==============================] - 2s 8ms/step - loss: 0.0270 - accuracy: 0.5730 - val_loss: 0.0481 - val_accuracy: 0.5477
Epoch 7/80
266/266 [==============================] - 2s 8ms/step - loss: 0.0194 - accuracy: 0.5761 - val_loss: 0.0350 - val_accuracy: 0.5508
Epoch

In [24]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = neural_network_1.evaluate(
    X_test_scaled_1,
    y_test_1, 
    verbose=2
)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

99/99 - 1s - loss: 0.0060 - accuracy: 0.5753 - 591ms/epoch - 6ms/step
Loss: 0.006004038266837597, Accuracy: 0.5753337740898132


In [25]:
# Make predictions for 10 season 3 hidden layer 
predictions_ten_seasons_1 = (neural_network_1.predict(X_test_scaled) > 0.5).astype("int32")

# Create a DataFrame to compare the predictions with the actual values
results_ten_seasons_1 = pd.DataFrame({"predictions": predictions_ten_seasons_1.ravel(), "actual": y_test})

# Display sample data
results_ten_seasons_1.sample(10)

NameError: name 'X_test_scaled' is not defined

In [ ]:
# plot the validation split to verify the model is not overfitting
history = neural_network_1.fit(
    X,
    y,
    verbose=0,
    validation_split=0.1,
    epochs=80,
    #batch_size=128
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy','validation accuracy'])

### Report for Part 1:
We have achieved an accuracy of about 57%! We originally had hoped for at least 40% accuracy. So right away, this first model is very promising. We hypothesize that testing the more recent five seasons on their own will have better results as sports can change drastically year-to-year. We will now move on to test those five more recent seasons with three hidden layers.

## Part 2: Neural Network with Deep Learning
### Five seasons of data from 2016 to 2021
### Three hidden layers with LeakyReLU activation and maximum nodes, one outer layer with linear activation, MSE as the loss activation, and a maximum of 130 epochs

In [ ]:
# Read in the 2016-2021 dataframe from the `read_csv_files.ipynb` file
# Drop the 'Date' column from the dataframe
five_seasons_nn = df_combined_2016_2021.drop(
    columns=['Date']
)

# Display the dataframe
five_seasons_nn

In [ ]:
# Review the data types
five_seasons_nn.dtypes

In [ ]:
# Create a list of the categorical variables
categorical_variables_2 = list(
    five_seasons_nn.dtypes[five_seasons_nn.dtypes == 'object'].index
)

# Display the list
categorical_variables_2

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(
    sparse = False
)

In [ ]:
# Encode the categorical variables into numerical format
encoded_data_2 = enc.fit_transform(
    five_seasons_nn[categorical_variables_2]
)

In [ ]:
# Create a dataframe with the encoded data
encoded_five_seasons_df = pd.DataFrame(
    encoded_data_2,
    columns = enc.get_feature_names_out(
        categorical_variables_2
    )
)

# Display the dataframe
encoded_five_seasons_df

In [ ]:
# Create a new variable for the exisiting numerical data
numerical_variables_2 = five_seasons_nn.drop(
    columns = categorical_variables_2
)

# Display the dataframe
numerical_variables_2

In [ ]:
# Add the numerical data to the categorical encoded data dataframe
encoded_five_seasons_df = pd.concat(
    [encoded_five_seasons_df,
     numerical_variables_2],
    axis=1
)

# Display the dataframe
encoded_five_seasons_df

In [ ]:
# Add a "Signal" column which tracks the "Win_Margin" column
encoded_five_seasons_df['Win_Margin_Signal'] = 0.0

# When the "Win_Margin" is a greater than or equal to 0, set signal to 1
encoded_five_seasons_df.loc[(encoded_five_seasons_df['Win_Margin'] >= 0), 'Win_Margin_Signal'] = 1

# When the "Win_Margin" is less than 0, set signal to -1
encoded_five_seasons_df.loc[(encoded_five_seasons_df['Win_Margin'] < 0), 'Win_Margin_Signal'] = -1

# Display the dataframe
encoded_five_seasons_df

In [ ]:
# Change the 'Win_Margin_Signal' column to float data type
encoded_five_seasons_df['Win_Margin_Signal'] = encoded_five_seasons_df['Win_Margin_Signal'].astype(float)

# Verify that the data types are correct
encoded_five_seasons_df.dtypes

In [ ]:
# Display the amount of each signal option
encoded_five_seasons_df['Win_Margin_Signal'].value_counts()

In [ ]:
# Set the y (target value) to 'Win_Margin_Signal' column
y = encoded_five_seasons_df['Win_Margin_Signal']

# Set the X (features) to all columns except 'Win_Margin_Signal'
X = encoded_five_seasons_df.drop(columns=['Win_Margin_Signal'])

# Display a sample of each dataframe
display(y[:5])
display(X.head())

In [ ]:
# Split the data into training and testing variables
X_train, X_test, y_train, y_test_2 = train_test_split(
    X,
    y,
    random_state=1
)

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled_2 = X_scaler.transform(X_train)
X_test_scaled_2 = X_scaler.transform(X_test)

In [ ]:
# Define the number of inputs to the model
inputs_2 = len(X_train.iloc[0])

# Display the amount of inputs
print(f"The amount of inputs is {inputs_2}.")

# Define the first hidden layer
hidden_2_layer_1 = ((inputs_2 + 1) // 2) + 33

# Display the amount of nodes
print(f"The amount of nodes in the first hidden layer is {hidden_2_layer_1}.")

# Define the second hidden layer
hidden_2_layer_2 = ((hidden_2_layer_1 + 1) // 2) + 10

# Display the amount of nodes
print(f"The amount of nodes in the second hidden layer is {hidden_2_layer_2}.")

# Define the third hidden layer
hidden_2_layer_3 = ((hidden_2_layer_2 + 1) // 2)

# Display the amount of nodes
print(f"The amount of nodes in the third hidden layer is {hidden_2_layer_3}.")

In [ ]:
# Create a Sequential mode instance
neural_network_2 = Sequential()

In [ ]:
# Add the hidden layers
# First layer
neural_network_2.add(
    Dense(
        units=hidden_2_layer_1,
        input_dim=inputs_2,
        activation='LeakyReLU'
    )
)

# Second layer
neural_network_2.add(
    Dense(
        units=hidden_2_layer_2,
        activation='LeakyReLU'
    )
)

# Third layer
neural_network_2.add(
    Dense(
        units=hidden_2_layer_3,
        activation='LeakyReLU'
    )
)

# Add the output layer
neural_network_2.add(
    Dense(
        units=outputs,
        activation='linear'
    )
)

In [ ]:
# Display the Sequential model summary
neural_network_2.summary()

In [ ]:
# Compile the Sequential model
neural_network_2.compile(
    loss="mse",
    optimizer="adam", 
    metrics=["accuracy"]
)

In [ ]:
# Fit the model using 130 epochs
fit_model = neural_network_2.fit(
    X_train_scaled_2,
    y_train,
    epochs=130,
    validation_split=0.1,
    callbacks=[early_stopping]
)

In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = neural_network_2.evaluate(
    X_test_scaled_2,
    y_test_2, 
    verbose=2
)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Make predictions for 5 season 3 hidden layers
predictions_five_seasons_1 = (neural_network_2.predict(X_test_scaled) > 0.5).astype("int32")

# Create a DataFrame to compare the predictions with the actual values
results_five_seasons_1 = pd.DataFrame({"predictions": predictions_five_seasons_1.ravel(), "actual": y_test})

# Display sample data
results_five_seasons_1.sample(10)

In [ ]:
# plot the validation split to verify the model is not overfitting
history = neural_network_2.fit(
    X,
    y,
    verbose=0,
    validation_split=0.1,
    epochs=130,
    #batch_size=128
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy','validation accuracy'])

## Report for Part 2:
We have achieved about 55% accuracy with only the five most recent seasons of data. This is very interesting. We had originally though that the more recent five seasons would have a better outcome as sports teams can change year-to-year. Either way, 55-57% accuracy is very good. We will now move on with the testing this data with four hidden layers instead of only three.

## Part 3: Neural Network with Deep Learning 
### 10 seasons of data from 2011 to 2021
### Four hidden layers with LeakyReLU activation and maximum nodes, one outer layer with linear activation, MSE as the loss activation, and a maximum of 100 epochs

In [ ]:
# Display the dataframe, reuse the encoded data with ten seasons
display(encoded_ten_seasons_df)

# Display the amount of each signal option
display(encoded_ten_seasons_df['Win_Margin_Signal'].value_counts())

In [ ]:
# Set the y (target value) to 'Win_Margin_Signal' column
y = encoded_ten_seasons_df['Win_Margin_Signal']

# Set the X (features) to all columns except 'Win_Margin_Signal'
X = encoded_ten_seasons_df.drop(columns=['Win_Margin_Signal'])

# Display a sample of each
display(y[:5])
display(X.head())

In [ ]:
# Split the data into training and testing variables
X_train, X_test, y_train, y_test_3 = train_test_split(
    X,
    y,
    random_state=1
)

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled_3 = X_scaler.transform(X_train)
X_test_scaled_3 = X_scaler.transform(X_test)

In [ ]:
# Display the amount of inputs
print(f"The amount of inputs is {inputs_1}.")

# Define the first hidden layer
hidden_3_layer_1 = ((inputs_1) // 2) + 23

# Display the amount of nodes
print(f"The amount of nodes in the first hidden layer is {hidden_3_layer_1}.")

# Define the second hidden layer
hidden_3_layer_2 = ((hidden_3_layer_1) // 2) + 11

# Display the amount of nodes
print(f"The amount of nodes in the second hidden layer is {hidden_3_layer_2}.")

# Define the third hidden layer
hidden_3_layer_3 = ((hidden_3_layer_2 + 1) // 2) + 4

# Display the amount of nodes
print(f"The amount of nodes in the third hidden layer is {hidden_3_layer_3}.")

# Define the fourth hidden layer
hidden_3_layer_4 = ((hidden_3_layer_3 + 1) // 2)

# Display the amount of nodes
print(f"The amount of nodes in the fourth hidden layer is {hidden_3_layer_4}.")

In [ ]:
# Create a Sequential model instance
neural_network_3 = Sequential()

In [ ]:
# Add the hidden layers
# First layer
neural_network_3.add(
    Dense(
        units=hidden_3_layer_1,
        input_dim=inputs_1,
        activation='LeakyReLU'
    )
)

# Second layer
neural_network_3.add(
    Dense(
        units=hidden_3_layer_2,
        activation='LeakyReLU'
    )
)

# Third layer
neural_network_3.add(
    Dense(
        units=hidden_3_layer_3,
        activation='LeakyReLU'
    )
)

# Fourth layer
neural_network_3.add(
    Dense(
        units=hidden_3_layer_4,
        activation='LeakyReLU'
    )
)

# Add the output layer
neural_network_3.add(
    Dense(
        units=outputs,
        activation='linear'
    )
)

In [ ]:
# Display the Sequential model summary
neural_network_3.summary()

In [ ]:
# Compile the Sequential model
neural_network_3.compile(
    loss="mse",
    optimizer="adam", 
    metrics=["accuracy"]
)

In [ ]:
# Fit the model using 100 epochs
fit_model = neural_network_3.fit(
    X_train_scaled_3,
    y_train,
    epochs=100,
    validation_split=0.1,
    callbacks=[early_stopping]
)

In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = neural_network_3.evaluate(
    X_test_scaled_3,
    y_test_3, 
    verbose=2
)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Make predictions for 10 season 
predictions_ten_seasons_2 = (neural_network_3.predict(X_test_scaled) > 0.5).astype("int32")

# Create a DataFrame to compare the predictions with the actual values
results_ten_seasons_2 = pd.DataFrame({"predictions": predictions_ten_seasons_2.ravel(), "actual": y_test})

# Display sample data
results_ten_seasons_2.sample(10)

In [ ]:
# plot the validation split to verify the model is not overfitting
history = neural_network_3.fit(
    X,
    y,
    verbose=0,
    validation_split=0.1,
    epochs=100,
    #batch_size=128
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy','validation accuracy'])

## Report for Part 3:
We have achieved about 57% accuracy. This is nearly the same result that we got when we tested the same 10 seasons with only three hidden layers. We do see a bit less loss in this model, though, which is a desirable difference. We will now move on to test the more recent five seasons of data with four hidden layers.

## Part 4: Neural Network with Deep Learning 
### Five seasons of data from 2016 to 2021
### Four hidden layers with LeakyReLU activation and maximum nodes, one outer layer with linear activation, MSE as the loss activation, and a maximum of 60 epochs

In [ ]:
# Display the dataframe, reuse the encoded data with five seasons
display(encoded_five_seasons_df)

# Display the amount of each signal option
display(encoded_five_seasons_df['Win_Margin_Signal'].value_counts())

In [ ]:
# Set the y (target value) to 'Win_Margin_Signal' column
y = encoded_five_seasons_df['Win_Margin_Signal']

# Set the X (features) to all columns except 'Win_Margin_Signal'
X = encoded_five_seasons_df.drop(columns=['Win_Margin_Signal'])

# Display a sample of each
display(y[:5])
display(X.head())

In [ ]:
# Split the data into training and testing variables
X_train, X_test, y_train, y_test_4 = train_test_split(
    X,
    y,
    random_state=1
)

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled_4 = X_scaler.transform(X_train)
X_test_scaled_4 = X_scaler.transform(X_test)

In [ ]:
# Display the amount of inputs
print(f"The amount of inputs is {inputs_2}.")

# Define the first hidden layer
hidden_4_layer_1 = ((inputs_2) // 2) + 23

# Display the amount of nodes
print(f"The amount of nodes in the first hidden layer is {hidden_4_layer_1}.")

# Define the second hidden layer
hidden_4_layer_2 = ((hidden_4_layer_1) // 2) + 11

# Display the amount of nodes
print(f"The amount of nodes in the second hidden layer is {hidden_4_layer_2}.")

# Define the third hidden layer
hidden_4_layer_3 = ((hidden_4_layer_2 + 1) // 2) + 4

# Display the amount of nodes
print(f"The amount of nodes in the third hidden layer is {hidden_4_layer_3}.")

# Define the fourth hidden layer
hidden_4_layer_4 = ((hidden_4_layer_3 + 1) // 2)

# Display the amount of nodes
print(f"The amount of nodes in the fourth hidden layer is {hidden_4_layer_4}.")

In [ ]:
# Create a Sequential model instance
neural_network_4 = Sequential()

In [ ]:
# Add the hidden layers
# First layer
neural_network_4.add(
    Dense(
        units=hidden_4_layer_1,
        input_dim=inputs_2,
        activation='LeakyReLU'
    )
)

# Second layer
neural_network_4.add(
    Dense(
        units=hidden_4_layer_2,
        activation='LeakyReLU'
    )
)

# Third layer
neural_network_4.add(
    Dense(
        units=hidden_4_layer_3,
        activation='LeakyReLU'
    )
)

# Fourth layer
neural_network_4.add(
    Dense(
        units=hidden_4_layer_4,
        activation='LeakyReLU'
    )
)

# Add the output layer
neural_network_4.add(
    Dense(
        units=outputs,
        activation='linear'
    )
)

In [ ]:
# Display the Sequential model summary
neural_network_4.summary()

In [ ]:
# Compile the Sequential model
neural_network_4.compile(
    loss="mse",
    optimizer="adam", 
    metrics=["accuracy"]
)

In [ ]:
# Fit the model using 60 epochs
fit_model = neural_network_4.fit(
    X_train_scaled_4,
    y_train,
    epochs=60,
    validation_split=0.1,
    callbacks=[early_stopping]
)

In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = neural_network_4.evaluate(
    X_test_scaled_4,
    y_test_4, 
    verbose=2
)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Make predictions for 10 season 
predictions_five_seasons_2 = (neural_network_4.predict(X_test_scaled) > 0.5).astype("int32")

# Create a DataFrame to compare the predictions with the actual values
results_five_seasons_2 = pd.DataFrame({"predictions": predictions_five_seasons_2.ravel(), "actual": y_test})

# Display sample data
results_five_seasons_2.sample(10)

In [ ]:
# plot the validation split to verify the model is not overfitting
history = neural_network_4.fit(
    X,
    y,
    verbose=0,
    validation_split=0.1,
    epochs=60,
    #batch_size=128
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy','validation accuracy'])

## Report for Part 4:
We have achieved about 55% accuracy. This is nearly the same result that we got when we tested the same five seasons with only three hidden layers. We also see about the same loss in this model, which is a difference from the 10-season testing where we saw a decrease in loss when a fourth hidden layer was added.

# The best model we created was model "neural_network_3" which used four hidden layers with maximum nodes, LeakyReLU activation, one output layer with linear activation, and 100 epochs. We will now use this model to predict outcomes for the 2021-2022 NBA Season and compare them with the actual outcomes of that season.